In [1]:
import pandas as pd
import numpy as np
import os
import igraph

In [2]:
cdr3info = pd.read_table("input_files/nlv-llw.txt")
cdr3graph = pd.read_table("input_files/nlv-llw.graph.txt")

In [3]:
cdr3info.head()

,cdr3aa,epitope,species,gene,v.segm,j.segm,v.end,j.start,cid
0,CAAADDKIIF,LLWNGPMAV,HomoSapiens,TRA,TRAV12-2*01,TRAJ30*01,2,4,h.A.LLWNGPMAV.1
1,CAAFDDKIIF,LLWNGPMAV,HomoSapiens,TRA,TRAV12-2*01,TRAJ30*01,2,4,h.A.LLWNGPMAV.1
2,CAAIDDKIIF,LLWNGPMAV,HomoSapiens,TRA,TRAV12-2*01,TRAJ30*01,2,4,h.A.LLWNGPMAV.1
3,CAAKDDKIIF,LLWNGPMAV,HomoSapiens,TRA,TRAV12-2*01,TRAJ30*01,2,4,h.A.LLWNGPMAV.1
4,CAALDDKIIF,LLWNGPMAV,HomoSapiens,TRA,TRAV12-2*01,TRAJ30*01,2,4,h.A.LLWNGPMAV.1


In [4]:
cdr3graph.head()

,species,gene,epitope,to.cdr3aa,from.cdr3aa
0,human,TRA,LLWNGPMAV,CAVSSDKIIF,CAVSGDKIIF
1,human,TRA,LLWNGPMAV,CAVSTDKLIF,CAVNTDKLIF
2,human,TRA,LLWNGPMAV,CAVTDDKIIF,CAVADDKIIF
3,human,TRA,LLWNGPMAV,CAVTDDKIIF,CAATDDKIIF
4,human,TRA,LLWNGPMAV,CAVSTDKLIF,CAVSPDKLIF


In [5]:
#assertion, that there is no cdr3 specific to two epitopes.
print(len(cdr3info))
print(len(cdr3info.drop_duplicates(subset=["cdr3aa", "v.segm"], keep="first")))

147
147


In [6]:
print(cdr3info.columns)

Index(['cdr3aa', 'epitope', 'species', 'gene', 'v.segm', 'j.segm', 'v.end',
       'j.start', 'cid'],
      dtype='object')


In [7]:
cdr3info.rename(columns={"cdr3aa":"from.cdr3aa", 
                         "v.segm":"from.v.segm", 
                         "j.segm":"from.j.segm",
                         "v.end":"from.v.end",
                         "j.start":"from.j.start"}, inplace=True)

In [8]:
len(cdr3graph)

1600

In [9]:
spec = {"human":"HomoSapiens"}
cdr3graph.replace({"species":spec}, inplace=True)
cdr3graph.head()

,species,gene,epitope,to.cdr3aa,from.cdr3aa
0,HomoSapiens,TRA,LLWNGPMAV,CAVSSDKIIF,CAVSGDKIIF
1,HomoSapiens,TRA,LLWNGPMAV,CAVSTDKLIF,CAVNTDKLIF
2,HomoSapiens,TRA,LLWNGPMAV,CAVTDDKIIF,CAVADDKIIF
3,HomoSapiens,TRA,LLWNGPMAV,CAVTDDKIIF,CAATDDKIIF
4,HomoSapiens,TRA,LLWNGPMAV,CAVSTDKLIF,CAVSPDKLIF


In [10]:
cdr3graph = cdr3graph.merge(cdr3info, on=["species", "gene", "epitope", "from.cdr3aa"], how="outer")

In [11]:
len(cdr3graph)

1600

In [12]:
cdr3info.rename(columns={"from.cdr3aa":"to.cdr3aa", 
                         "from.v.segm":"to.v.segm", 
                         "from.j.segm":"to.j.segm",
                         "from.v.end":"to.v.end",
                         "from.j.start":"to.j.start"}, inplace=True)

In [13]:
cdr3graph = cdr3graph.merge(cdr3info, on=["species", "gene", "epitope", "to.cdr3aa", "cid"], how="outer")

In [14]:
len(cdr3graph)

1600

In [15]:
print(cdr3graph.head())

       species gene    epitope   to.cdr3aa  from.cdr3aa  from.v.segm  \
0  HomoSapiens  TRA  LLWNGPMAV  CAVSSDKIIF   CAVSGDKIIF  TRAV12-2*01   
1  HomoSapiens  TRA  LLWNGPMAV  CAVSSDKIIF   CAVSDDKIIF  TRAV12-2*01   
2  HomoSapiens  TRA  LLWNGPMAV  CAVSSDKIIF   CAVGSDKIIF  TRAV12-2*01   
3  HomoSapiens  TRA  LLWNGPMAV  CAVTGDKIIF   CAVSGDKIIF  TRAV12-2*01   
4  HomoSapiens  TRA  LLWNGPMAV  CAVTGDKIIF  CAVTGDDKIIF  TRAV12-2*01   

  from.j.segm  from.v.end  from.j.start              cid    to.v.segm  \
0   TRAJ30*01         3.0           5.0  h.A.LLWNGPMAV.1  TRAV12-2*01   
1   TRAJ30*01         3.0           4.0  h.A.LLWNGPMAV.1  TRAV12-2*01   
2   TRAJ30*01         3.0           5.0  h.A.LLWNGPMAV.1  TRAV12-2*01   
3   TRAJ30*01         3.0           5.0  h.A.LLWNGPMAV.1  TRAV12-2*01   
4   TRAJ30*01         3.0           5.0  h.A.LLWNGPMAV.1  TRAV12-2*01   

   to.j.segm  to.v.end  to.j.start  
0  TRAJ30*01       3.0         5.0  
1  TRAJ30*01       3.0         5.0  
2  TRAJ30*01     

In [16]:
def tuple_maker(df, tuplecol, *values):
    def gettuple(*values):
        return sorted(*values)
    
    df[tuplecol] = df.apply(lambda col: gettuple(col[val] for val in values), axis=1)
    return df

In [17]:
cdr3graph = tuple_maker(cdr3graph, "tuple", "to.cdr3aa", "from.cdr3aa")

cdr3graph_missing = cdr3graph[cdr3graph["v.segm"].isna()]
cdr3graph_missing.to_csv("input_files/nlv-llw.missing_values.txt", 
sep="\t", 
index=None)

cdr3info["cid"].unique()

cdr3graph = tuple_maker(cdr3graph, 
"tuple", 
"to.cdr3aa", 
"from.cdr3aa")

graph = igraph.Graph.TupleList(list(cdr3graph["tuple"]))

vertices = graph.vs['name']

vertices_unique = list(cdr3info.drop_duplicates(subset=["cid"], 
keep="first")["to.cdr3aa"])

vertices_unique_positions = np.where(np.isin(vertices, vertices_unique))[0]

print(list((vertices[i] in vertices_unique) for i in vertices_unique_positions))

edge_distances = graph.shortest_paths_dijkstra()

for i in vertices_unique_positions:
    edges = np.array(edge_distances[i])
    print(len(np.where(~np.isin(edges, np.inf))[0]))

In [18]:
cdr3graph_work = cdr3graph[~(cdr3graph["cid"].isna())].reset_index(drop=True)

In [19]:
print(len(cdr3graph_work), #check num of edges
      
      #check num of unique edges in undirected graph;
      len(cdr3graph_work["tuple"].str.join("|").drop_duplicates(keep="first")),
      
      #check num of unique edges #in directed graph)
      len(cdr3graph_work.drop_duplicates(subset=["to.cdr3aa",
                                                 "from.cdr3aa"],
                                         keep="first")))

556 278 556


In [20]:
def bfs_from_minmean(cluster_graph):
    work_graph = pd.DataFrame()
    vertices = cluster_graph.vs['name']
    edge_distances = cluster_graph.shortest_paths_dijkstra()#mode="ALL")
    mean_distances = [np.mean(i) for i in edge_distances]
    
    vertix_with_minmean = np.argmin(mean_distances)
    
    checked_vertices = [vertix_with_minmean]
    
    while len(checked_vertices) != len(vertices):
        for vertix in checked_vertices:
            edges = edge_distances[vertix]
            nb_vertices = list(np.argwhere(np.array(edges) == 1)[:, 0])
            for new_vertix in nb_vertices:
                if new_vertix in checked_vertices:
                    continue

                from_vertix = vertices[vertix]
                to_vertix = vertices[new_vertix]
                work_graph = work_graph.append(cdrs[(cdrs["from.cdr3aa"] == from_vertix) & (
                        cdrs["to.cdr3aa"] == to_vertix)])
                checked_vertices.append(new_vertix)
            
    return work_graph
    

In [21]:
sorted_graph = pd.DataFrame()
for cluster, cdrs in cdr3graph_work.groupby("cid"):
    cluster_graph = igraph.Graph.TupleList(list(cdrs["tuple"]))
    sorted_graph = sorted_graph.append(bfs_from_minmean(cluster_graph))

In [22]:
sorted_graph = sorted_graph.reset_index(drop=True)
sorted_graph.to_csv("nlv-llw.graphpath.txt", sep="\t", index=None, columns=sorted_graph.columns[:-1])

Now we have sorted graph.

Time for central vertices: we just need to get first vertices for each cluster in sorted_graph

In [23]:
minmeanvertices = sorted_graph.drop_duplicates(subset="cid", keep="first")
minmeanvertices

,species,gene,epitope,to.cdr3aa,from.cdr3aa,from.v.segm,from.j.segm,from.v.end,from.j.start,cid,to.v.segm,to.j.segm,to.v.end,to.j.start,tuple
0,HomoSapiens,TRA,LLWNGPMAV,CAVSDDKIIF,CAVTDDKIIF,TRAV12-2*01,TRAJ30*01,3.0,4.0,h.A.LLWNGPMAV.1,TRAV12-2*01,TRAJ30*01,3.0,4.0,"[CAVSDDKIIF, CAVTDDKIIF]"
49,HomoSapiens,TRA,LLWNGPMAV,CAGSNARLMF,CAVSNARLMF,TRAV12-2*01,TRAJ31*01,3.0,4.0,h.A.LLWNGPMAV.13,TRAV12-2*01,TRAJ31*01,2.0,4.0,"[CAGSNARLMF, CAVSNARLMF]"
55,HomoSapiens,TRA,NLVPMVATV,CASPTGNQFYF,CASNTGNQFYF,"TRAV24*01,TRAV18*01",TRAJ49*01,2.0,3.0,h.A.NLVPMVATV.15,TRAV24*01,TRAJ49*01,2.0,4.0,"[CASNTGNQFYF, CASPTGNQFYF]"
65,HomoSapiens,TRA,NLVPMVATV,CFLDNNNDMRF,CILDNNNDMRF,TRAV26-2*01,TRAJ43*01,3.0,4.0,h.A.NLVPMVATV.51,TRAV26-2*01,TRAJ43*01,-1.0,4.0,"[CFLDNNNDMRF, CILDNNNDMRF]"
82,HomoSapiens,TRB,LLWNGPMAV,CATSGGGAYEQYF,CATSRGGAYEQYF,TRBV15*01,TRBJ2-7*01,5.0,8.0,h.B.LLWNGPMAV.14,TRBV15*01,TRBJ2-7*01,4.0,8.0,"[CATSGGGAYEQYF, CATSRGGAYEQYF]"
86,HomoSapiens,TRB,LLWNGPMAV,CASSAGTGAYEQYF,CASSPGTGAYEQYF,TRBV27*01,TRBJ2-7*01,4.0,9.0,h.B.LLWNGPMAV.3,TRBV9*01,TRBJ2-7*01,4.0,9.0,"[CASSAGTGAYEQYF, CASSPGTGAYEQYF]"
104,HomoSapiens,TRB,NLVPMVATV,CASSYQTGASYGYTF,CASSYQTGAIYGYTF,TRBV6-5*01,TRBJ1-2*01,5.0,10.0,h.B.NLVPMVATV.32,"TRBV6-3*01,TRBV6-1*01,TRBV6-5*01",TRBJ1-2*01,5.0,10.0,"[CASSYQTGAIYGYTF, CASSYQTGASYGYTF]"
116,HomoSapiens,TRB,NLVPMVATV,CASSLPPGATNEKLFF,CASSLAPGATNEKLFF,"TRBV7-9*01,TRBV6-1*01,TRBV7-8*01,TRBV7-6*01,TR...",TRBJ1-4*01,5.0,9.0,h.B.NLVPMVATV.6,TRBV7-6*01,TRBJ1-4*01,5.0,9.0,"[CASSLAPGATNEKLFF, CASSLPPGATNEKLFF]"


In [24]:
minmeanvertices[minmeanvertices["from.v.end"]<0]

,species,gene,epitope,to.cdr3aa,from.cdr3aa,from.v.segm,from.j.segm,from.v.end,from.j.start,cid,to.v.segm,to.j.segm,to.v.end,to.j.start,tuple


#Uh oh
#Strange! Let's take a look in segments.aaparts.txt


In [25]:
#segment_parts = pd.read_table("segments.aaparts.txt")
#segment_parts.head()

,species,gene,cdr3,type,segm
0,HomoSapiens,TRA,C,V,TRAV26-1
1,HomoSapiens,TRA,CA,V,TRAV24
2,HomoSapiens,TRA,CAA,V,TRAV13-1
3,HomoSapiens,TRA,CAAS,V,TRAV29/DV5
4,HomoSapiens,TRA,CAE,V,TRAV5


In [29]:
#strange_tcr = minmeanvertices[minmeanvertices["from.v.end"]<0]
#strange_cdr3 = list(strange_tcr["from.cdr3aa"])[0]
#strange_vsegm = list(strange_tcr["from.v.segm"])[0]
#strange_specie = list(strange_tcr["species"])[0]
#print(strange_cdr3, strange_vsegm)

In [30]:
#sliced_cdr3 = strange_cdr3
#while len(sliced_cdr3) != 0:
#    if len(segment_parts[(segment_parts["species"] == strange_specie) &
#                         (segment_parts["cdr3"] == sliced_cdr3) & 
#                         (segment_parts["segm"]+"*01" == strange_vsegm)]):
#        break
#    sliced_cdr3 = sliced_cdr3[:-1]
#print(sliced_cdr3, len(sliced_cdr3))

#Anyway, let's create aminoacid sequences of selected tcrs.

In [31]:
import gene_to_protein
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

In [32]:
tcr_n_constructor = gene_to_protein.tcr_nucleotide_constructor()
tcr_a_constructor = gene_to_protein.tcr_aminoacid_constructor()

In [34]:
with open("nlv-llw.sequences_out.txt", "w") as out:
    #out.write("cid\tV.segm.seq\tCDR3\tJ.segm.seq\tC.segm.seq\ttype\tV.segm\tJ.segm\tspecie\n")
    for index, tcr in minmeanvertices.iterrows():
        #In the future I'll implement tcr_construction as function on the columns
        v_genes = tcr["from.v.segm"].split(',')
        j_genes = tcr["from.j.segm"].split(',')
        cdr3 = tcr["from.cdr3aa"]
        specie = tcr["species"]
        for v_gene in v_genes:
            for j_gene in j_genes:
                print(tcr["from.v.end"], tcr["from.j.start"], cdr3, v_genes, j_genes)
                
                v_end = tcr["from.v.end"]
                if v_end < 0:
                    v_end = 1
                j_start = tcr["from.j.start"]
                if j_start <0:
                    print(j_start)
                if v_end > j_start:
                    j_start = v_end
                
                tcr_nucleseq = tcr_n_constructor.get_tcr(cdr3, specie, int(v_end), int(j_start), v_gene, j_gene, add_c=True, seqlengths=True)
                tcr_aminoseq = tcr_a_constructor.get_tcr(cdr3, specie, v_gene, j_gene, add_c=True, seqlengths=True)
                
                nucleseq = tcr_nucleseq[0]
                tcr_nucle_lengths = list(tcr_nucleseq[1:])
                #row = [tcr["cid"]]
                v_end = tcr["from.v.end"]
                if v_end < 0:
                    v_end = 1
                j_start = tcr["from.j.start"]
                if j_start <0:
                    print(j_start)
                out.write(">{}|{}|{}|{}|{}|{}|\n".format(tcr["cid"],
                                                      v_gene, 
                                                      j_gene, 
                                                      specie, 
                                                      "nucleotide", 
                                                      cdr3))
                row=[]
                for i in range(1, len(tcr_nucle_lengths)):
                    tcr_nucle_lengths[i] += tcr_nucle_lengths[i-1]
                tcr_nucle_lengths[0] -= int(v_end)*3
                tcr_nucle_lengths[1] += (len(cdr3)-int(j_start))*3
                for i in range(len(tcr_nucle_lengths)):
                    if i == 0:
                        print(str(Seq(nucleseq[:tcr_nucle_lengths[i]]).translate()))
                        row.append(nucleseq[:tcr_nucle_lengths[i]].replace("N", "?"))
                    else:
                        print(str(Seq(nucleseq[:tcr_nucle_lengths[i]]).translate()))
                        row.append(nucleseq[tcr_nucle_lengths[i-1]:tcr_nucle_lengths[i]].replace("N", "?"))
                #row += ["nucleotide", v_gene, j_gene, specie]
                out.write("{}\n\n".format("\t".join(row)))
                
                
                aminoseq = tcr_aminoseq[0]
                tcr_amino_lengths = list(tcr_aminoseq[1:])
                #row = [tcr["cid"]]
                row = []
                out.write(">{}|{}|{}|{}|{}|{}|\n".format(tcr["cid"],
                                                      v_gene, 
                                                      j_gene, 
                                                      specie, 
                                                      "aminoacid", 
                                                      cdr3))
                for i in range(len(tcr_amino_lengths)):
                    if i == 0:
                        row.append(aminoseq[:tcr_amino_lengths[i]])
                    else:
                        tcr_amino_lengths[i] += tcr_amino_lengths[i-1]
                        row.append(aminoseq[tcr_amino_lengths[i-1]:tcr_amino_lengths[i]])
                #row += ["aminoacid", v_gene, j_gene, specie]
                out.write("{}\n\n\n".format("\t".join(row)))

                #print(">{}\n{}\n{}\n{}\n".format(tcr["cid"], tcr_nucleseq[0], tcr_aminoseq[0], str(Seq(tcr_nucleseq[0], generic_dna).translate())))

3.0 4.0 CAVTDDKIIF ['TRAV12-2*01'] ['TRAJ30*01']
QKEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPELIMFIYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYL
QKEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPELIMFIYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYLCAVXDDKIIF
QKEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPELIMFIYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYLCAVXDDKIIFGKGTRLHILP
QKEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPELIMFIYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYLCAVXDDKIIFGKGTRLHILPNIQNPDPAVYQLRDSKSSDKSVCLFTDFDSQTNVSQSKDSDVYITDKTVLDMRSMDFKSNSAVAWSNKSDFACANAFNNSIIPEDTFFPSPESSCDVKLVEKSFETDTNLNFQNLSVIGFRILLLKVAGFNLLMTLRLWSS*
3.0 4.0 CAVSNARLMF ['TRAV12-2*01'] ['TRAJ31*01']
QKEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPELIMFIYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYL
QKEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPELIMFIYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYLCAVXNARLMF
QKEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPELIMFIYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYLCAVXNARLMFGDGTQLVVKP
QKEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPELIMFIYSNGDKE

/Users/alex/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/Seq.py:2309: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


Check for mistakes:

In [53]:
#out.write("cid\tV.segm.seq\tCDR3\tJ.segm.seq\tC.segm.seq\ttype\tV.segm\tJ.segm\tspecie\n")
for index, tcr in sorted_graph.iterrows():
    #In the future I'll implement tcr_construction as function on the columns
    v_genes = tcr["from.v.segm"].split(',')
    j_genes = tcr["from.j.segm"].split(',')
    cdr3 = tcr["from.cdr3aa"]
    specie = tcr["species"]
    for v_gene in v_genes:
        for j_gene in j_genes:
            v_end = tcr["from.v.end"]
            if v_end < 0:
                v_end = 1
            j_start = tcr["from.j.start"]
            if j_start <0:
                print(j_start)
            if v_end > j_start:
                j_start = v_end
            
            tcr_nucleseq = tcr_n_constructor.get_tcr(cdr3, specie, int(v_end), int(j_start), v_gene, j_gene, add_c=True, seqlengths=True)
            tcr_aminoseq = tcr_a_constructor.get_tcr(cdr3, specie, v_gene, j_gene, add_c=True, seqlengths=True)

            nucleseq = tcr_nucleseq[0]
            tcr_nucle_lengths = list(tcr_nucleseq[1:])
            #row = [tcr["cid"]]
            
            aminoseq = tcr_aminoseq[0]
            tcr_amino_lengths = list(tcr_aminoseq[1:])
            
            
            for i in range(1, len(tcr_nucle_lengths)):
                tcr_nucle_lengths[i] += tcr_nucle_lengths[i-1]
            lenseq = 0
            for i in range(len(tcr_nucle_lengths)):
                if i == 0:
                    seqtrans = str(Seq(nucleseq[:tcr_nucle_lengths[i]]).translate())
                elif i == 3:
                    seqtrans = str(Seq(nucleseq[tcr_nucle_lengths[i-1]-1:tcr_nucle_lengths[i]]).translate()).replace("*", "")
                    #lenseq -= 1
                else:
                    seqtrans = str(Seq(nucleseq[tcr_nucle_lengths[i-1]:tcr_nucle_lengths[i]]).translate())
                
                seqamino = aminoseq[lenseq:lenseq+len(seqtrans)]
                lenseq += len(seqtrans)
                if (i != 1 and seqtrans != seqamino):
                    print(tcr["from.v.end"], tcr["from.j.start"], cdr3, v_gene, j_gene)
                    print("Error in {} part".format({0:"V", 1:"CDR3", 2:"J", 3:"C"}[i]))
                    print("{}\n{}\n{}\n\n".format(seqtrans, seqamino, seqtrans==seqamino))
                    
            #row += ["nucleotide", v_gene, j_gene, specie]

            #row += ["aminoacid", v_gene, j_gene, specie]

            #print(">{}\n{}\n{}\n{}\n".format(tcr["cid"], tc

5.0 10.0 CASSYQTGASYGYTF TRBV6-1*01 TRBJ1-2*01
Error in V part
NAGVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYYSASEGTTDKGEVPNGYNVSRLNKREFSLRLESAAPSQTSVYFCASSE
NAGVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYYSASEGTTDKGEVPNGYNVSRLNKREFSLRLESAAPSQTSVYFCASSY
False


5.0 10.0 CASSYQTGASYGYTF TRBV6-1*01 TRBJ1-2*01
Error in V part
NAGVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYYSASEGTTDKGEVPNGYNVSRLNKREFSLRLESAAPSQTSVYFCASSE
NAGVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYYSASEGTTDKGEVPNGYNVSRLNKREFSLRLESAAPSQTSVYFCASSY
False


5.0 10.0 CASSYQTGASYGYTF TRBV6-1*01 TRBJ1-2*01
Error in V part
NAGVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYYSASEGTTDKGEVPNGYNVSRLNKREFSLRLESAAPSQTSVYFCASSE
NAGVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYYSASEGTTDKGEVPNGYNVSRLNKREFSLRLESAAPSQTSVYFCASSY
False


5.0 9.0 CASSLAPGATNEKLFF TRBV6-1*01 TRBJ1-4*01
Error in V part
NAGVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYYSASEGTTDKGEVPNGYNVSRLNKREFSLRLESAAPSQTSVYFCASSE
NAGVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYYSASE

/Users/alex/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/Seq.py:2309: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)
